In [1]:
import pandas as pd
import requests
from cssselect import GenericTranslator
from lxml.etree import XPath # delete
from lxml import html
from bs4 import BeautifulSoup # delete
import wget
from zipfile import ZipFile
import fnmatch
import os
import random 
from time import sleep

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

In [3]:
path = os.getcwd()

In [4]:
# User Agent
headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36",
}

In [5]:
# Wikipedia site with social revolts records
url = "https://en.wikipedia.org/wiki/List_of_revolutions_and_rebellions"

In [6]:
respuesta = requests.get(url, headers=headers)

In [9]:
# HTLM tree parser

parser = html.fromstring(respuesta.text)


## This is working fine.
### Here I parse the text of a given decade block from the wikipedia page using *xpath*

In [ ]:
# ingles = parser.get_element_by_id('/html/body/div[3]/div[3]/div[5]/div[1]/ul[22]')
ingles = parser.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/ul[22]')
for i in ingles:
    print (i.text_content())

### This is working fine too. A different method with css selectors. Here I parse the text of a given decade block from the wikipedia page using *css selectors.*

In [ ]:
year = parser.xpath(GenericTranslator().css_to_xpath("ul:nth-child(100) li"))
for text in year:
    print(text.text_content())

## Finidng the links
### here you can find the links for a given decade

In [ ]:
links = parser.xpath(GenericTranslator().css_to_xpath('ul:nth-child(100) a'))
for link in links:
#     print(link.text_content())
    print('https://en.wikipedia.org' + link.attrib['href'])

# Use another wikipedia page source 

## I look for a more simple page to scrap the dataset


In [11]:
url_2 = 'https://en.wikipedia.org/wiki/List_of_protests_in_the_21st_century'

In [12]:
respuesta_2 = requests.get(url_2, headers=headers)

In [13]:
# HTLM tree parser

parser_2 = html.fromstring(respuesta_2.text)


In [ ]:
wiki_data = parser.xpath(GenericTranslator().css_to_xpath("ul:nth-child(100) li"))
for text in wiki_data:
    print(text.text_content())

## This is working fine.
### Here I parse the text of a given decade block from the wikipedia page using *xpath*

protest = parser_2.xpath(GenericTranslator().css_to_xpath('dt'))
for text in protest:
    print(text.text_content())
    print()

## Extract socioeconomical data from the worlbank

### Create a Function that make a call to the API and return a data frame

In [7]:
#####################

# This function make a specific dataseries request to the worlbank API and return a pandas df as output. 
# It takes 3  arguments:
# 1. Indicator Code "indic_code", 
# 2. year range "year_s", 
# 3. Path destination "destination"

#####################


def API_call_to_df(indic_code, 
                   year_s = '2000:2020', 
                   destination = path + "/" + "WB_datasets"):
    WB_url_seed = 'http://api.worldbank.org/v2/country/all/indicator/' + indic_code
    my_params = {'date' : year_s,
                'incomelevel' :'',
                'downloadformat' : 'csv',
                'per_page' : '304'}
    
    def number_of_cols(year_s): # Define function to asses index of columns to retrieve
        my_years_int = year_s.split(":")
        my_colums_indx = [*range(0, list(map(int, my_years_int))[1] - list(map(int, my_years_int))[0] + 5)]
        return(my_colums_indx)
    
    for file in os.listdir(destination):
        if fnmatch.fnmatch(file, "API_" + indic_code.upper() + "*"):
            os.remove(destination + "/" + file)
            response = requests.get(WB_url_seed, params = my_params)
            my_zip_file = wget.download(response.url, out= destination)
            print("file exists and will be replaced")
            break
            
    else:
        response = requests.get(WB_url_seed, params = my_params)
        my_zip_file = wget.download(response.url, out= destination)
        
    with ZipFile(my_zip_file, 'r') as zipObj:
            for content in zipObj.namelist():
                if fnmatch.fnmatch(content, 'API_*'):
                    #print(content) #wihtin the content of the zip file find and extract the csv file that contain the data
                    my_filename = content
                    zipObj.extract(content, path = destination)
                    my_df = pd.read_csv(destination + '/' + my_filename, 
                                                header = 2,
                                                usecols = number_of_cols(year_s))
                    os.remove(destination + "/" + my_filename)
                    
    return(my_df)


In [22]:
wer_indicator = API_call_to_df(indic_code="NY.GNP.MKTP.PP.CD", year_s = '2010:2020')
wer_indicator.head(5)

BadZipFile: File is not a zip file

In [44]:
wer_indicator.shape

(264, 24)

## Importing the Worldbank data catalog
We see that the catalog contain > 1500 series!!!

In [9]:
full_db_catalog = pd.read_excel(io=path + "/" + "WDI_CETS.xls",
             sheet_name = 1)

full_db_catalog

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors",Environment,Agricultural production,NaN,NaN
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer production),Environment,Agricultural production,NaN,NaN
2,AG.CON.FERT.ZS,Fertilizer consumption (kilograms per hectare of arable land),Environment,Agricultural production,NaN,NaN
3,AG.LND.AGRI.K2,Agricultural land (sq. km),Environment,Land use,NaN,NaN
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Environment,Land use,NaN,NaN
...,...,...,...,...,...,...
1499,WP15163_4.1,Mobile account (% age 15+),Financial Sector,Access,NaN,NaN
1500,WP15163_4.2,"Mobile account, male (% age 15+)",Financial Sector,Access,NaN,NaN
1501,WP15163_4.3,"Mobile account, female (% age 15+)",Financial Sector,Access,NaN,NaN
1502,WP15163_4.8,"Mobile account, income, poorest 40% (% ages 15+)",Financial Sector,Access,NaN,NaN


## Create a Helper Function to find easily a given catalog by code or by name

In [48]:
def find_your_serie(in_your_serie, full_wb_catalog = full_db_catalog,  by_code = False, by_name = False):
    if by_code:
        
        return(full_db_catalog[full_db_catalog['Series Code'].str.contains(in_your_serie, case = False)])
    
    if by_name:
        
        return(full_db_catalog[full_db_catalog['Series Name'].str.contains(in_your_serie, case = False)])
    else:
        
        return(full_db_catalog)
        

### Filtering the series that contain only variables exressed as %
We obtain only 755 variables which is a lot less!

In [57]:
raw_percentage_db_catalog = full_db_catalog[full_db_catalog['Series Name'].str.contains("%")]
raw_percentage_db_catalog

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...,Environment,Agricultural production,NaN,NaN
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Environment,Land use,NaN,NaN
7,AG.LND.ARBL.ZS,Arable land (% of land area),Environment,Land use,NaN,NaN
9,AG.LND.CROP.ZS,Permanent cropland (% of land area),Environment,Land use,NaN,NaN
11,AG.LND.EL5M.RU.ZS,Rural land area where elevation is below 5 met...,Environment,Land use,NaN,NaN
...,...,...,...,...,...,...
1499,WP15163_4.1,Mobile account (% age 15+),Financial Sector,Access,NaN,NaN
1500,WP15163_4.2,"Mobile account, male (% age 15+)",Financial Sector,Access,NaN,NaN
1501,WP15163_4.3,"Mobile account, female (% age 15+)",Financial Sector,Access,NaN,NaN
1502,WP15163_4.8,"Mobile account, income, poorest 40% (% ages 15+)",Financial Sector,Access,NaN,NaN


## Finding Economic indicator "GNI, PPP" code catalog

In [32]:
# full_db_catalog[full_db_catalog['Series Name'].str.contains("GNI, PPP")]
find_your_serie(by_name=True, full_wb_catalog=full_db_catalog, in_your_serie = "GNI, PPP")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
839,NY.GNP.MKTP.PP.CD,"GNI, PPP (current international $)",Economic Policy & Debt,Purchasing power parity,NaN,NaN
840,NY.GNP.MKTP.PP.KD,"GNI, PPP (constant 2011 international $)",Economic Policy & Debt,Purchasing power parity,NaN,NaN


## Finding Economic indicator "Corruption perception Index" code catalog

In [33]:
find_your_serie(by_name=True, full_wb_catalog=full_db_catalog, in_your_serie = "corruption")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
629,IQ.CPA.TRAN.XQ,"CPIA transparency, accountability, and corruption in the public sector rating (1=low to 6=high)",Public Sector,Policy & institutions,NaN,NaN


## Finding Economic indicator "Government Effectiveness" code catalog

In [38]:
find_your_serie(by_name=True, full_wb_catalog=full_db_catalog, in_your_serie = "governm")


,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
325,DT.ODA.ODAT.XP.ZS,Net ODA received (% of central government expense),Economic Policy & Debt,Official development assistance,NaN,NaN
463,FM.AST.CGOV.ZG.M3,Claims on central government (annual growth as % of broad money),Financial Sector,Assets,NaN,NaN
480,FS.AST.CGOV.GD.ZS,"Claims on central government, etc. (% GDP)",Financial Sector,Assets,NaN,NaN
487,GC.DOD.TOTL.CN,"Central government debt, total (current LCU)",Public Sector,Government finance,Deficit & financing,NaN
488,GC.DOD.TOTL.GD.ZS,"Central government debt, total (% of GDP)",Public Sector,Government finance,Deficit & financing,NaN
563,IC.GOV.DURS.ZS,Time spent dealing with the requirements of government regulations (% of senior management time),Private Sector & Trade,Business environment,NaN,NaN
667,MS.MIL.XPND.ZS,Military expenditure (% of central government expenditure),Public Sector,Defense & arms trade,NaN,NaN
669,NE.CON.GOVT.CD,General government final consumption expenditure (current US$),Economic Policy & Debt,National accounts,US$ at current prices,Expenditure on GDP
670,NE.CON.GOVT.CN,General government final consumption expenditure (current LCU),Economic Policy & Debt,National accounts,Local currency at current prices,Expenditure on GDP
671,NE.CON.GOVT.KD,General government final consumption expenditure (constant 2010 US$),Economic Policy & Debt,National accounts,US$ at constant 2010 prices,Expenditure on GDP


## Finding Economic indicator "Civil liberty index" code catalog

In [57]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "lib")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3


## Finding Economic indicator "Enviromental" code catalog

In [58]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "envi")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
609,IQ.CPA.BREG.XQ,CPIA business regulatory environment rating (1=low to 6=high),Public Sector,Policy & institutions,NaN,NaN
612,IQ.CPA.ENVR.XQ,CPIA policy and institutions for environmental sustainability rating (1=low to 6=high),Public Sector,Policy & institutions,NaN,NaN


## Finding Economic indicator "Expectancy" code catalog

In [60]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "life")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1109,SH.MMR.RISK,Lifetime risk of maternal death (1 in: rate varies by country),Health,Reproductive health,NaN,NaN
1110,SH.MMR.RISK.ZS,Lifetime risk of maternal death (%),Health,Reproductive health,NaN,NaN
1333,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",Health,Mortality,NaN,NaN
1334,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",Health,Mortality,NaN,NaN
1335,SP.DYN.LE00.MA.IN,"Life expectancy at birth, male (years)",Health,Mortality,NaN,NaN


## Finding Economic indicator "health" code catalog

In [61]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "health")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1048,SG.DMK.ALLD.FN.ZS,"Women participating in the three decisions (own health care, major household purchases, and visiting family) (% of women age 15-49)",Gender,Public life & decision making,NaN,NaN
1101,SH.MED.CMHW.P3,"Community health workers (per 1,000 people)",Health,Health systems,NaN,NaN
1121,SH.STA.ARIC.ZS,ARI treatment (% of children under 5 taken to a health provider),Health,Disease prevention,NaN,NaN
1123,SH.STA.BRTC.ZS,Births attended by skilled health staff (% of total),Health,Reproductive health,NaN,NaN
1155,SH.XPD.EXTR.ZS,External resources for health (% of total expenditure on health),Health,Health systems,NaN,NaN
1156,SH.XPD.OOPC.TO.ZS,Out-of-pocket health expenditure (% of total expenditure on health),Health,Health systems,NaN,NaN
1157,SH.XPD.OOPC.ZS,Out-of-pocket health expenditure (% of private expenditure on health),Health,Health systems,NaN,NaN
1158,SH.XPD.PCAP,Health expenditure per capita (current US$),Health,Health systems,NaN,NaN
1159,SH.XPD.PCAP.PP.KD,"Health expenditure per capita, PPP (constant 2011 international $)",Health,Health systems,NaN,NaN
1160,SH.XPD.PRIV.ZS,"Health expenditure, private (% of GDP)",Health,Health systems,NaN,NaN


## Finding Economic indicator "health" code catalog

In [65]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "health")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1048,SG.DMK.ALLD.FN.ZS,"Women participating in the three decisions (own health care, major household purchases, and visiting family) (% of women age 15-49)",Gender,Public life & decision making,NaN,NaN
1101,SH.MED.CMHW.P3,"Community health workers (per 1,000 people)",Health,Health systems,NaN,NaN
1121,SH.STA.ARIC.ZS,ARI treatment (% of children under 5 taken to a health provider),Health,Disease prevention,NaN,NaN
1123,SH.STA.BRTC.ZS,Births attended by skilled health staff (% of total),Health,Reproductive health,NaN,NaN
1155,SH.XPD.EXTR.ZS,External resources for health (% of total expenditure on health),Health,Health systems,NaN,NaN
1156,SH.XPD.OOPC.TO.ZS,Out-of-pocket health expenditure (% of total expenditure on health),Health,Health systems,NaN,NaN
1157,SH.XPD.OOPC.ZS,Out-of-pocket health expenditure (% of private expenditure on health),Health,Health systems,NaN,NaN
1158,SH.XPD.PCAP,Health expenditure per capita (current US$),Health,Health systems,NaN,NaN
1159,SH.XPD.PCAP.PP.KD,"Health expenditure per capita, PPP (constant 2011 international $)",Health,Health systems,NaN,NaN
1160,SH.XPD.PRIV.ZS,"Health expenditure, private (% of GDP)",Health,Health systems,NaN,NaN


## Finding Economic indicator "seats" code catalog

In [66]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "seat")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1049,SG.GEN.PARL.ZS,Proportion of seats held by women in national parliaments (%),Gender,Public life & decision making,NaN,NaN


## Finding Economic indicator "seats" code catalog

In [66]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "seat")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1049,SG.GEN.PARL.ZS,Proportion of seats held by women in national parliaments (%),Gender,Public life & decision making,NaN,NaN


## Finding Economic indicator "poverty" code catalog

In [71]:
# find_your_serie(by_code=True, full_wb_catalog=full_db_catalog, in_your_serie = "ff")
find_your_serie(by_name=True, in_your_serie = "povert")

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
1172,SI.POV.2DAY,Poverty headcount ratio at $3.10 a day (2011 PPP) (% of population),Poverty,Poverty rates,NaN,NaN
1173,SI.POV.DDAY,Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population),Poverty,Poverty rates,NaN,NaN
1174,SI.POV.GAP2,Poverty gap at $3.10 a day (2011 PPP) (%),Poverty,Poverty rates,NaN,NaN
1175,SI.POV.GAPS,Poverty gap at $1.90 a day (2011 PPP) (%),Poverty,Poverty rates,NaN,NaN
1177,SI.POV.NAGP,Poverty gap at national poverty lines (%),Poverty,Poverty rates,NaN,NaN
1178,SI.POV.NAHC,Poverty headcount ratio at national poverty lines (% of population),Poverty,Poverty rates,NaN,NaN
1179,SI.POV.RUGP,Rural poverty gap at national poverty lines (%),Poverty,Poverty rates,NaN,NaN
1180,SI.POV.RUHC,Rural poverty headcount ratio at national poverty lines (% of rural population),Poverty,Poverty rates,NaN,NaN
1181,SI.POV.URGP,Urban poverty gap at national poverty lines (%),Poverty,Poverty rates,NaN,NaN
1182,SI.POV.URHC,Urban poverty headcount ratio at national poverty lines (% of urban population),Poverty,Poverty rates,NaN,NaN


In [143]:
raw_percentage_db_catalog.head(10).sort_values(by = "Topic")#[raw_percentage_db_catalog[]]

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
29,BG.GSR.NFSV.GD.ZS,Trade in services (% of GDP),Economic Policy & Debt,Balance of payments,Current account,"Goods, services & income"
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer production),Environment,Agricultural production,NaN,NaN
4,AG.LND.AGRI.ZS,Agricultural land (% of land area),Environment,Land use,NaN,NaN
7,AG.LND.ARBL.ZS,Arable land (% of land area),Environment,Land use,NaN,NaN
9,AG.LND.CROP.ZS,Permanent cropland (% of land area),Environment,Land use,NaN,NaN
11,AG.LND.EL5M.RU.ZS,Rural land area where elevation is below 5 meters (% of total land area),Environment,Land use,NaN,NaN
13,AG.LND.EL5M.UR.ZS,Urban land area where elevation is below 5 meters (% of total land area),Environment,Land use,NaN,NaN
14,AG.LND.EL5M.ZS,Land area where elevation is below 5 meters (% of total land area),Environment,Land use,NaN,NaN
16,AG.LND.FRST.ZS,Forest area (% of land area),Environment,Land use,NaN,NaN
17,AG.LND.IRIG.AG.ZS,Agricultural irrigated land (% of total agricultural land),Environment,Land use,NaN,NaN


In [138]:
raw_percentage_db_catalog.columns.to_list()

['Series Code', 'Series Name', 'Topic', 'SubTopic1', 'SubTopic2', 'SubTopic3']

### Display the number of dataseries per topic

In [141]:
raw_percentage_db_catalog.sort_values(by = "Topic").groupby(['Topic'], as_index = False).size()#[raw_percentage_db_catalog[]]

Topic
Economic Policy & Debt       105
Education                    124
Environment                   68
Financial Sector              35
Gender                         9
Health                       131
Infrastructure                 6
Poverty                       20
Private Sector & Trade        91
Public Sector                 27
Social Protection & Labor    139
dtype: int64

#### We Observe that the topics that contain the most number of series are:

1. **Social Protection & Labor**
2. **Health**
3. **Education**
4. **Economic Policy & Debt**

## Explore the various topics to asess quality of the data

In [254]:
raw_percentage_db_catalog[raw_percentage_db_catalog['Topic'] == "Social Protection & Labor"].head(10)

,Series Code,Series Name,Topic,SubTopic1,SubTopic2,SubTopic3
869,per_allsp.adq_pop_tot,Adequacy of social protection and labor programs (% of total welfare of beneficiary households),Social Protection & Labor,Performance,NaN,NaN
870,per_allsp.ben_q1_tot,Benefit incidence of social protection and labor programs to poorest quintile (% of total SPL benefits),Social Protection & Labor,Performance,NaN,NaN
871,per_allsp.cov_pop_tot,Coverage of social protection and labor programs (% of population),Social Protection & Labor,Performance,NaN,NaN
872,per_lm_alllm.adq_pop_tot,Adequacy of unemployment benefits and ALMP (% of total welfare of beneficiary households),Social Protection & Labor,Performance,NaN,NaN
873,per_lm_alllm.ben_q1_tot,Benefit incidence of unemployment benefits and ALMP to poorest quintile (% of total U/ALMP benefits),Social Protection & Labor,Performance,NaN,NaN
874,per_lm_alllm.cov_pop_tot,Coverage of unemployment benefits and ALMP (% of population),Social Protection & Labor,Performance,NaN,NaN
875,per_lm_alllm.cov_q1_tot,Coverage of unemployment benefits and ALMP in poorest quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
876,per_lm_alllm.cov_q2_tot,Coverage of unemployment benefits and ALMP in 2nd quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
877,per_lm_alllm.cov_q3_tot,Coverage of unemployment benefits and ALMP in 3rd quintile (% of population),Social Protection & Labor,Performance,NaN,NaN
878,per_lm_alllm.cov_q4_tot,Coverage of unemployment benefits and ALMP in 4th quintile (% of population),Social Protection & Labor,Performance,NaN,NaN


In [281]:
wer_indicator.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
2000              263
2001              263
2002              261
2003              261
2004              259
2005              251
2006              245
2007              244
2008              235
2009              232
2010              230
2011              230
2012              225
2013              240
2014              234
2015              231
2016              235
2017              245
2018              258
2019              263
2020              264
dtype: int64

In [317]:
# Count the number of NaNs in a given series df
# Indicator_Code = 
# wer_indicator
years = pd.DataFrame(wer_indicator.isnull().sum()).reset_index()[4:]
years.columns = ("year", "NaNs")
years.head(6)

,year,NaNs
4,2000,263
5,2001,263
6,2002,261
7,2003,261
8,2004,259
9,2005,251


### Make a function that take a dataframe and return the NaNs listed by year in all countries

In [353]:
def data_quality_asses(dataframe):
    df_years = pd.DataFrame(dataframe.isnull().sum()).reset_index()[4:] # Obtain Nas per year
    df_years.columns = ("year", "NaNs") # rename columns
    df_years["Indicator Code"] = dataframe["Indicator Code"][0] # add new column with the code name
    df_years.set_index( ["Indicator Code"], inplace = True) # reset index to the code name
    df_years = pd.pivot(df_years, columns= 'year', values= "NaNs") # pivot the dataset
    return (df_years)

In [355]:
data_quality_asses(wer_indicator)[:]

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Indicator Code,,,,,,,,,,,,,,,,,,,,,
per_allsp.adq_pop_tot,263,263,261,261,259,251,245,244,235,232,...,230,225,240,234,231,235,245,258,263,264
